# Analysis of the South Korea COVID-19 Kaggle Dataset

<img src="static/virus.png" height='500px' width='1000px'>

[Picture source](http://www.lowellma.gov/coronavirus) (02.04.2020)

## Introduction
The COVID-19 virus is an incisive event in the 21st century. Nearly all countries in the world are affected and the effects are noticeable on a macroscopic level for countries, provinces and cities and on a microscopic level for individuals. From its origin in the Chinese province of Wuhan the virus spread all over the world. The question is what effects and what extensions of the effects are to be anticipated by countries, regions and individuals that have not experienced the full possible extend of the virus. Data Science can give answers to those questions on the basis of data that has been collected in regions close to the origin of the virus.

## The Data

Kaggle provides a very comprehensive [Dataset for South Korea](https://www.kaggle.com/kimjihoo/coronavirusdataset/data)
 (Version 31.03.2020), a country close to the origin and with good data quality that will be used as basis for the data science analysis. The goal is to analyse the data with respect to parameters that are transferable and universaly applicable, gain insights into the progression of the virus for regions and indiviuals and try to find corresponding models.


## Let's open the folder and have a first look at the data

### Imports

In [ ]:
import os
import pandas as pd 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
for file in os.listdir('coronavirusdataset/'):
    print(file)

So there seem to be quied some files in csv format. Lets load them into a dict and print the head.

In [ ]:
df_dict={}
for file in os.listdir('coronavirusdataset/'):
    df=pd.read_csv('coronavirusdataset/'+file)
    df_dict[file.split('.')[0]]=df
    print(file.split('.')[0])
    print(df.head())

Here is a tabularized description of the data sets

File| Description | 
--- | --- 
Case| Data of COVID-19 Infection Caseses | 
PatientInfo| Epidemiological data of COVID-19 patients in South Korea | 
PatientRoute| Route data of COVID-19 patients in South Korea | 
Region| Location and statistical data of the regions in South Korea| 
SearchTrend| Trend data of the keywords searched in NAVER which is one of the largest portals in South Korea | 
SeoulFloating| Data of floating population in Seoul, South Korea (from SK Telecom Big Data Hub) | 
Time| Time series data of COVID-19 status in South Korea | 
TimeAge| Time series data of COVID-19 status in terms of the age in South Korea | 
TimeGender| Time series data of COVID-19 status in terms of gender in South Korea | 
TimeProvince| Time series data of COVID-19 status in terms of the Province in South Korea | 
Weather| Data of the weather in the regions of South Korea | 

We immediately see, that data is very specific to South Korea. The demographic information is given in form of the city names and provinces, that are non transferable quantities, like population or population density. Therefore we need to add those informations ourself. Thank god the internet provides us with credible sites like [City Population](https://citypopulation.de/), with a standartized data format that we can easily exctract data from. We are especially interested in the overall population and the population density.

<img src="static/citypopulation_table.png" height='500px' width='1000px'>

To get the information for all cities listed in the TimeProvince dataset one could either search the cities individually or write a web scraper to get the information from the City Population website. Since the last approach is more fun let's do just just that, using [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) a web scraping library and [Selenium](https://selenium-python.readthedocs.io/) a web driver library, both can be installed via pip. I am not going to go to explain the code in detail, but what basically happens:

- Selenium allows an automatic search of a city
- Loop overall results and make sure, that city name and province are correct
- Open the link with the match and use BeautifulSoup to extract the standardized html table
- With a regex operation the two numbers for the population and the population density are extracted
- The results are saved in an excel file

### You don't have to run the code, the result is provided in the Region_with_population_population_density.xlsx file. If you want to run it, you have to download a browser driver (I used the driver for Firefox, called [geckodriver](https://github.com/mozilla/geckodriver/releases) and put the absolute path to the driver_path variable in the code


In [ ]:
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm

df=df_dict['Region']

#absolute path to webdriver, I used the Firefox driver geckodriver for mac os: https://github.com/mozilla/geckodriver/releasesdriver_path='/absolute path to webdriver

driver=webdriver.Firefox(executable_path =driver_path)

for i in tqdm(range(df.shape[0])):
    match=False
    city=df['city'][i]
    province=df['province'][i]
    
    try:
        #Go to Citypopulation search
        driver.get("https://www.citypopulation.de/search.html")
        time.sleep(2)
        #Search for the city

        search_country= driver.find_element_by_id("countries1")
        search_country.send_keys('South Korea')
        search_place= driver.find_element_by_id("places1")
        search_place.send_keys(city)
        search_place.send_keys(Keys.RETURN)
        time.sleep(3)

        #Get result element
        results=driver.find_elements_by_class_name("result")
        #check results, since a lot of the cities have the same name we also consider the province, since for a metropolitan citiy the province is the city itself it gets its on case
        for result in results:
            if province in result.text:
                match=True
                result.click()
                break
            elif 'Metropolitan  City'  in result.text:
                match=True
                result.click()
                break
    

        if match==True:
            time.sleep(3)
            #Scrap the html
            URL=driver.current_url

            response=requests.get(URL)
            soup=BeautifulSoup(response.text,'html.parser')

            #Get the table with pupultion information
            table=soup.find('table',{"id":"ts"}).tbody
            rows = table.find_all("td")
            
            #Scrap population density and populatin number from table
            df.at[i,'PopulationDensity']=float(re.search(r'data-density="(.*?)"', str(rows[0])).group(1))
            df.at[i,'Population']=float(re.search(r'">(.*?)</td>', str(rows[-2])).group(1).replace(',',''))
            time.sleep(1)
        else:
          print(f'No match for {city} in {province}')  
    except:
        print(f'Did not work for {city} in {province}')

driver.close()
driver.quit()

df.to_excel('Region_with_population_population_density.xlsx')
print(df)

With the exception of Yeoju-si the scraper worked correctly. The values for the cities that could not be found were added manually with a help of a quick Google search.

## Let's start with the acutal analysis
Lets try to get an understanding about the progression of virus. Therefore, we take a look at the "TimeAge" data and try to find out who is affected in risk and possible in danger. Therefore we are going to look at the overall cumulative numbers of conformations and deceased cases and calculate the motility rate. We are going to use [plotly](https://plotly.com/) for this, a fantastic, ease to use library that allows for interactive plots.

In [ ]:
df=df_dict['TimeAge']
fig = make_subplots(rows=3, cols=1, subplot_titles=("Number of cumulative conformations", "Number of cumulative deceased", "Mortality rate"))
colors=plt.cm.Blues(np.linspace(0, 1, len(df['age'].unique())))

for i, age in enumerate(df['age'].unique()):
    plt_df=df[df['age']==age]
    x=plt_df['date'].values
    y_conf=plt_df['confirmed'].values
    
    y_des=plt_df['deceased'].values

    #decease rate
    y_des_rate=y_des/y_conf*100

    color=f"rgba({colors[i][0]},{colors[i][1]},{colors[i][2]},{colors[i][3]})"
    
    
    fig.add_trace(go.Scatter(x=x, y=y_conf, name=age, line=dict(color=color)), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=y_des, name=age, line=dict(color=color), showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=x, y=y_des_rate, name=f"{age} cumulative", line=dict(color=color), showlegend=False), row=3, col=1)

fig.update_yaxes(title_text="[-]", row=1, col=1)
fig.update_yaxes(title_text="[-]", row=2, col=1)
fig.update_yaxes(title_text="[%]", row=3, col=1)
fig.update_layout(height=800,
                  width=600,
                  title_text="Analysis of COVID-19 progression over time",
                  template="plotly_white")
fig.show()

From this graph we can constate a couple of interesting things:
- Younger people tend to have a higher infection rate, especially people in their 20s
- Older people above 70 show the highest mortality rate, between 5-10 %
- The cumulative curves show a flattening trend, which indicates, that spread of the virus is getting contained


Let's investigate if gender has an effect on the virus. Therefore, we take a look at the "TimeGender" data set and produce a plot of the mortality rate similar as above

In [ ]:
df=df_dict['TimeGender']
fig = go.Figure()
colors=plt.cm.Blues(np.linspace(0, 1, len(df['sex'].unique())))

for i, sex in enumerate(df['sex'].unique()):
    plt_df=df[df['sex']==sex]
    x=plt_df['date'].values
    y_conf=plt_df['confirmed'].values
    
    y_des=plt_df['deceased'].values

    #decease rate
    y_des_rate=y_des/y_conf*100

    color=f"rgba({colors[i][0]},{colors[i][1]},{colors[i][2]},{colors[i][3]})"
    
    
    fig.add_trace(go.Scatter(x=x, y=y_des_rate, name=f"{sex} cumulative cases", line=dict(color=color), showlegend=True))

fig.update_yaxes(title_text="[%]")
fig.update_layout(height=400,
                  width=600,
                  title_text="Analysis of the gender influence on the mortality rate",
                  template="plotly_white")
fig.show()

Ok.... this does not look good for men; on average the mortality rate is twice as high as for women and both rates seem still to increase.

Now we want to investigate the influence of demographics and infrastructure of the cities. Therefore, we read in the "Region" data with the added information about population and population density for the cities and fuse the data with the "Case" data.

In [ ]:
# Sum all cases for one city
unique_cities=df_dict['Case'][['province','city']].drop_duplicates()

df_sum=pd.DataFrame([])
for i, j in enumerate(unique_cities.index):
    city=unique_cities['city'][j]
    province=unique_cities['province'][j]
    df_sum.at[i,'province']=province
    df_sum.at[i,'city']=city
    idx_cases=np.where((df_dict['Case']['city']==city) & (df_dict['Case']['province']==province),True,False)
    sum_i=np.sum(df_dict['Case'].loc[idx_cases,'confirmed'].values)
    df_sum.at[i,'confirmed']=sum_i
print(df_sum.head())

In [ ]:
import copy
df_dict['Region']=pd.read_excel('Region_with_population_population_density.xlsx')

df=copy.deepcopy(df_sum)

for idx in df.index:
    city=df['city'][idx]
    province=df['province'][idx]
    #Fuse with data from region data set
    for key in ['Population','PopulationDensity','kindergarten_count','elementary_school_count','university_count','academy_ratio','elderly_population_ratio','elderly_alone_ratio','nursing_home_count']:
        idx_region=np.where((df_dict['Region']['city']==city) & (df_dict['Region']['province']==province),True,False)

        value=df_dict['Region'].loc[idx_region,key].values 
        #only append if value exit
        if value:
            df.at[idx,key]=value

print(df.head())


Now let's do a quick correlation matrix to see if demographics and infrastructure influence the cases of confirmation

In [ ]:
import plotly.figure_factory as ff

corr_matrix=df.corr()
y_keys=list(corr_matrix.index)
corr_matrix=corr_matrix['confirmed'].to_frame()
fig = ff.create_annotated_heatmap(corr_matrix.values[1:].round(2), colorscale='Blues', x=list(corr_matrix.columns),y=y_keys[1:])
fig.update_layout(
                  title_text="Correlation matrix for confirmation number with parameters of infrastructure and demografics",
                  template="plotly_white")
fig.show()

So the demographics and infrastructure do not seem to influence the number of confirmations very much. The development of a model based on those features is therefore not reasonable. Let's check if the number of confirmations is related to certain events and to locations. Therefore, we investigate the infection cases with respect to the number of corresponding confirmed cases.

In [ ]:
df=df_dict['Case']
df_cases_sorted_sum=df.groupby('infection_case').sum().confirmed


fig = go.Figure(data=[go.Pie(labels=df_cases_sorted_sum.index, values=df_cases_sorted_sum)])
fig.update_layout(height=800,
                  width=800,
                  title_text="Number of confirmations",
                  template="plotly_white")
fig.show()



The diagram demonstrates, that the biggest cause for conformations are public places/events. Let's see if the number of conformations is related to a certain city/region. We therefore plot the geographical data of call cases and display the number of conformations by relation of the size of the maker in red. We also display the city names and locations in black.

In [ ]:
plot_df=df_dict['Case']

fig=go.Figure()
for i in range(plot_df.shape[0]):
    try:
        fig.add_trace(go.Scattergeo(
                lon = [float(plot_df['longitude'][i])],
                lat = [float(plot_df['latitude'][i])],
                #hovertext=[f"Case {plot_df['infection_case']}"],
                #hoverinfo="text",
                text = f"Case {plot_df['infection_case'][i]} {plot_df['confirmed'][i]} confirmed cases",
                name = f"{plot_df['city'][i]} {plot_df['province'][i]}",
                marker = dict(size = plot_df['confirmed'][i]/20, color = 'red', line_width = 0, opacity=0.5),
                showlegend=False))
        fig.add_trace(go.Scattergeo(
                lon = [float(plot_df['longitude'][i])],
                lat = [float(plot_df['latitude'][i])],
                #hovertext=[f"Case {plot_df['infection_case']}"],
                name= f"{plot_df['confirmed'][i]} confirmed cases",
                text = f"{plot_df['city'][i]}",
                mode='markers+text',
                marker = dict(size = 1, color = 'black', line_width = 0, opacity=1),
                showlegend=False))
    except:
        pass


fig.update_layout(height=1000,
                  width=1000,
                  title_text="Geografical summary of all cases",
                  template="plotly_white",
                  geo = go.layout.Geo(
                    resolution = 50,
                    scope = 'asia',
                    showframe = False,
                    showcoastlines = True,
                    landcolor = "rgb(229, 229, 229)",
                    countrycolor = "grey" ,
                    coastlinecolor = "white",
                    projection_type = 'mercator',
                    showsubunits=True, subunitcolor="Blue",
                    lonaxis_range= [ 125, 130 ],
                    lataxis_range= [ 33, 39.0 ],
                    domain = dict(x = [ 0, 1 ], y = [ 0, 1 ])
                ),)

fig.show()

One can observe, that the most conformations occurred in the Daegu region, especially the city Nam-gu, do to a public event of the Sincheonji Curch. This underlines the previous observation of the influence of public events, the infection in crowds, but also the fact that the South Korean government seems to acted very quickly to prevent the virus from spreading to other cities and regions. This might also explain the flattening of the cumulative number of confirmation cases.


The last thing we want to get insight into, are the effects and the progression of the virus for the individuals. We therefore look the "PatientInfo" data and try to investigate the influence of sex, age on the average time of recovering and deceasing.

In [ ]:
df=df_dict['PatientInfo']
df['recovering_time']=pd.to_datetime(df.loc[df['state']=='released']['released_date'])-pd.to_datetime(df.loc[df['state']=='released']['confirmed_date'])
df['deceasing_time']=pd.to_datetime(df.loc[df['state']=='deceased']['deceased_date'])-pd.to_datetime(df.loc[df['state']=='deceased']['confirmed_date'])

Let's take a look a the time for recovering and deceasing

In [ ]:
fig = go.Figure()
val_rec=df[df['recovering_time'].notnull()]['recovering_time']

val_dec=df[df['deceasing_time'].notnull()]['deceasing_time']

y_rec=[int(i.split(' days')[0]) for i in val_rec.astype(str).values if int(i.split(' days')[0])>0]
fig.add_trace(go.Box(y=y_rec, boxpoints='all', name='Time until release',
            jitter=0.3,pointpos=0))
y_dec=[int(i.split(' days')[0]) for i in val_dec.astype(str).values if int(i.split(' days')[0])>0]
fig.add_trace(go.Box(y=y_dec, boxpoints='all', name='Time until decease', 
            jitter=0.3,pointpos=0))

fig.update_yaxes(title_text="[days]")
fig.update_layout(height=400,
                  width=600,
                  title_text="Box plot analyis time for release and decease",
                  template="plotly_white")
fig.show()


In [ ]:
age_list=['0s','10s','20s','30s','40s','50s','60s','70s']
sex_list=['male','female']

fig = make_subplots(rows=1, cols=2, subplot_titles=("Influence of age and sex on the recovering time", "Influence of age and sex of the deceasing time"))

for age in age_list:
    val_rec=df[df['age']==age]['recovering_time'].dropna()
    val_dec=df[df['age']==age]['deceasing_time'].dropna()
    y_rec=[int(i.split(' days')[0]) for i in val_rec.astype(str).values if int(i.split(' days')[0])>0]
    y_dec=[int(i.split(' days')[0]) for i in val_dec.astype(str).values if int(i.split(' days')[0])>0]
    fig.add_trace(go.Box(y=y_rec, boxpoints='all', name=age, marker_color = 'blue',
            jitter=0.3,pointpos=0), row=1 , col=1)
    fig.add_trace(go.Box(y=y_dec, boxpoints='all', name=age, marker_color = 'blue',
            jitter=0.3,pointpos=0), row=1 , col=2)

for sex in sex_list:
    val_rec=df[df['sex']==sex]['recovering_time'].dropna()
    val_dec=df[df['sex']==sex]['deceasing_time'].dropna()
    y_rec=[int(i.split(' days')[0]) for i in val_rec.astype(str).values if int(i.split(' days')[0])>0]
    y_dec=[int(i.split(' days')[0]) for i in val_dec.astype(str).values if int(i.split(' days')[0])>0]
    fig.add_trace(go.Box(y=y_rec, boxpoints='all', name=sex, marker_color = 'orange',
            jitter=0.3,pointpos=0), row=1 , col=1)
    fig.add_trace(go.Box(y=y_dec, boxpoints='all', name=sex, marker_color = 'orange',
            jitter=0.3,pointpos=0, showlegend=False), row=1 , col=2)

fig.update_yaxes(title_text="[days]", row=1, col=1)
fig.update_yaxes(title_text="[days]", row=1, col=2)
fig.update_layout(height=600,
                  width=1000,
                  template="plotly_white")

The graphs show not trend nor truths that we have not already observed early. Again, the high risk and deceasing rate is illustrated for people of higher age by the graph for the deceasing time, but trends for recovering time or the deceasing time with respect to age or sex cannot be observed.

All in all, we can conclude:
- The virus can infect everybody, no matter what age or sex
- It is far more dangerous for older people with a age of 60 or higher
- The mortality rate is strongly correlated with age and sex, being the highest for old males
- The spread of the virus is (at least in South Korea) not correlated with transferable features for demographics and infrastructure like population density and number of kinder gardens etc. the training of a model that is transferable for other countries is therefore not possible
- The virus is spread mostly in public events
- The time of recovering or the time until the decease of individuals is not related to age or sex
- For South Korea the overall progression of the virus is flattening, probably due to very fast and restrictive actions after a person has either been identified with the virus, or has have contact with a infected one


# Thank's for reading!